# Scraping data from subreddit r/AskDocs

In [12]:
import praw
import pandas as pd
from datasets import Dataset

In [3]:
reddit=praw.Reddit(user_agent=True,
                   client_secret='zS6cuhXCP0b1HCU0aSXl236vu9o9SA',client_id='OzzaZ-aY_0oo4TnMg0fnWA',username='EastLeft',password='randomaccnt278')

In [4]:
%%time
subreddit = reddit.subreddit('askdocs')
# Define lists to store data
data = []

# Scraping posts & Comments
for post in subreddit.top(limit= 100): 
    data.append({
        'Type': 'Post',
        'Post_id': post.id,
        'Title': post.title,
        'Author': post.author.name if post.author else 'Unknown',
        'Timestamp': post.created_utc,
        'Text': post.selftext,
        'Score': post.score,
        'Total_comments': post.num_comments,
        'Post_URL': post.url
    })

# Check if the post has comments
    if post.num_comments > 0:
        # Scraping comments for each post
        post.comments.replace_more(limit= 5)
        for comment in post.comments.list():
            data.append({
                'Type': 'Comment',
                'Post_id': post.id,
                'Title': post.title,
                'Author': comment.author.name if comment.author else 'Unknown',
                'Timestamp': pd.to_datetime(comment.created_utc, unit='s'),
                'Text': comment.body,
                'Score': comment.score,
                'Total_comments': 0, #Comments don't have this attribute
                'Post_URL': None  #Comments don't have this attribute
            })

# Create pandas DataFrame for posts and comments
df = pd.DataFrame(data)


CPU times: user 4.58 s, sys: 312 ms, total: 4.89 s
Wall time: 3min 51s


In [5]:
df.describe()

,Score,Total_comments
count,15152.000000,15152.000000
mean,54.582959,1.506402
std,586.542462,50.730565
min,-352.000000,0.000000
25%,2.000000,0.000000
50%,6.000000,0.000000
75%,21.000000,0.000000
max,66976.000000,5868.000000


In [15]:
df

,Type,Post_id,Title,Author,Timestamp,Text,Score,Total_comments,Post_URL
0,Post,ju4mua,41yo suboxone patient with lung cancer. I don'...,medthrowaway87ei,1605371446.0,Hello all. I hope this update doesn't break an...,66976,5868,https://www.reddit.com/r/AskDocs/comments/ju4m...
1,Comment,ju4mua,41yo suboxone patient with lung cancer. I don'...,theothernguyen,2020-11-14 18:04:06,as a doctor who often treats patients such as ...,2966,0,None
2,Comment,ju4mua,41yo suboxone patient with lung cancer. I don'...,keyst,2020-11-14 17:06:42,You aren’t alone - all of us are with you righ...,3115,0,None
3,Comment,ju4mua,41yo suboxone patient with lung cancer. I don'...,scorpio6519,2020-11-14 18:10:12,Hi Robert. I've read all your posts. I am so s...,703,0,None
4,Comment,ju4mua,41yo suboxone patient with lung cancer. I don'...,my-other-throwaway90,2020-11-14 17:27:51,I want you to know that I just three away my c...,1635,0,None
...,...,...,...,...,...,...,...,...,...
15147,Comment,hem4tr,My neighbor (36M) dropped dead in front of me....,pale_blue_moon,2020-06-24 21:04:23,Then why is alcohol legal and advertised?,2,0,None
15148,Comment,hem4tr,My neighbor (36M) dropped dead in front of me....,MD_Cosemtic,2020-06-24 18:30:38,>every death is a tragedy to medical professio...,11,0,None
15149,Comment,hem4tr,My neighbor (36M) dropped dead in front of me....,Gamestoreguy,2020-06-24 23:09:07,"Hey man, just wanted to let you know that you’...",10,0,None
15150,Comment,hem4tr,My neighbor (36M) dropped dead in front of me....,MD_Cosemtic,2020-06-25 03:09:08,Thank you,3,0,None
